# Cartera

Autores:
>Santiago Osorio Duque 

>Johan S. Méndez

## Objetivo:
Identificar si un registro necesita gestión de cobro con mora a mayor de 30 días

## Necesidad:
Poder tomar acciones tempranas y realizar las respectivas gestiones a los registros que necesitarán dado una predicción de mora de 30 días. Esto con el objetivo de reducir costos además de evitar la reducción de liquidez

## Análisis de información 
Se realizará limpieza de los datos que fueron suministrados. 
1. Comportamiento variables consignadas en la base de datos 
2. Encontrar relaciones entre ellas
3. Evolución a través del tiempo 

In [67]:
# Modulos escritos
from connect_database import connect

# Librerias 
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Se hace la lectura de la base de datos utilizando el módulo _connect_ que se encuentra escrito dentro del archivo __connect_database.py__. Con esto se establece la conexión a las bases guardadas en azure. A continuación se listan las bases a las cuales se quiere hacer la conexión 

In [3]:
server   = "carterasvr.database.windows.net"
database = "cartera"
username = "consulta"
password = "D4t4b1z2.123"

# Se establece la conexión 
cnxn = connect(server, database, username, password)

Petición SQL a la base de datos. Se almacena dentro de un marco de datos de Pandas

In [4]:
sql = ("""SELECT HIST_CARTERA.*, REGIONES.ID_REGION_NATURAL, REGIONES.NOMBRE AS REGION
FROM HIST_CARTERA LEFT JOIN
     (SELECT DIM_GEOGRAFIA.ID_GEOGRAFIA, DIM_GEOGRAFIA.ID_REGION_NATURAL, DRN.NOMBRE
FROM DIM_GEOGRAFIA
LEFT JOIN DIM_REGION_NATURAL DRN on DIM_GEOGRAFIA.ID_REGION_NATURAL = DRN.ID_REGION_NATURAL) REGIONES
ON HIST_CARTERA.ID_GEOGRAFIA = REGIONES.ID_GEOGRAFIA""")

data = pd.read_sql(sql, cnxn)


In [5]:
data

,ID_FECHA_CONSULTADA,ANIO_CARTERA,MES_CARTERA,ANIOMES_CARTERA,NUMERO_CONTRATO,TIPO_IDENTIFICACION,IDENTIFICACION,SEXO,INGRESO,REPUTACION_CLIENTE,...,CANTIDAD_FACTURAS_CON_MORA,FECHA_REGISTRO_CARTERA,FECHA_COBRO,FECHA_PAGO,FECHA_RECAUDO,ID_CONTRATO,ID_CLIENTE,ID_GEOGRAFIA,ID_REGION_NATURAL,REGION
0,20181201.0,2018,11,201811,758589809482,5,30567722,F,2454284.0,4.0,...,0.0,2018-11-23,2018-12-08,2018-11-26,2018-11-27,8689.0,3929.0,54670.0,2.0,Centro Oriente
1,20190101.0,2018,12,201812,830910315567,2,11642038,M,5492814.0,5.0,...,0.0,2018-12-23,2019-01-07,2018-12-27,2018-12-28,11372.0,5614.0,54670.0,2.0,Centro Oriente
2,20190101.0,2018,12,201812,915320322651,3,72850794,M,5946697.0,3.0,...,0.0,2018-12-23,2019-01-07,2018-12-24,2018-12-25,7070.0,6182.0,5480.0,4.0,Eje Cafetero - Antioquia
3,20190101.0,2018,12,201812,754486263805,5,71620041,F,5489184.0,5.0,...,0.0,2018-12-23,2019-01-07,2019-01-01,2019-01-02,10940.0,2145.0,5480.0,4.0,Eje Cafetero - Antioquia
4,20190201.0,2019,01,201901,482290887380,1,46788645,F,5005455.0,5.0,...,0.0,2019-01-23,2019-02-07,2019-01-27,2019-01-28,12811.0,3294.0,5040.0,4.0,Eje Cafetero - Antioquia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182774,20150801.0,2015,07,201507,396932713330,3,28314231,M,1806612.0,4.0,...,0.0,2015-07-23,2015-08-07,2015-07-25,2015-07-26,6678.0,10605.0,25426.0,2.0,Centro Oriente
182775,20150801.0,2015,07,201507,442296544038,4,78176770,M,609600.0,4.0,...,0.0,2015-07-23,2015-08-07,2015-07-23,2015-07-24,5952.0,8766.0,27205.0,6.0,Pacífico
182776,20150801.0,2015,07,201507,700030420895,1,68350115,M,2177628.0,5.0,...,0.0,2015-07-23,2015-08-07,2015-07-30,2015-07-31,12480.0,9006.0,81300.0,5.0,Llano
182777,20150801.0,2015,07,201507,919730907694,2,7059347,F,1539082.0,4.0,...,0.0,2015-07-23,2015-08-07,2015-08-04,2015-08-05,8604.0,797.0,52256.0,6.0,Pacífico


### Campos Agregados 

Inclusión del primer campo __FECHA_CONSULTADA__, la fecha viene por defecto en la base de datos con el formato (__20181201.0__). Para el manejo de datos se cambia el formato para el tipo (__2018-12-01__) en el formato _datetime_ de pandas

In [12]:
data["FECHA_CONSULTADA"] = pd.to_datetime(data["ID_FECHA_CONSULTADA"].astype("int").astype("str"), format = "%Y-%m-%d")

Segundo campo agregado __DIAS_CONOCIMIENTO_FACTURA__ consigna los dias de diferencia entre el registro de cartera y la consulta, es decir, cuantos días el usuario conoce su retraso. Además se normaliza para que se tenga en el campo de datos sin unidades y como punto flotante

In [21]:
data["DIAS_CONOCIMIENTO_FACTURA"] = (data["FECHA_CONSULTADA"] - data["FECHA_REGISTRO_CARTERA"])/np.timedelta64(1, "D")

Tercer campo agregado __DIAS_MORA__. Se puede inferir con los campos de la base los días de mora del usuario, si el resultado es un valor negativo se hizo el pago antes del registro de la fecha de cobro. Se pueden presentar varios casos , pero es importante conocer la forma en que el banco hace el cobro para poder determinar si en realidad la mora se relaciona con atraso o con una mala política de cobro del banco.

Para los valores perdidos se van a utilizar una columna transitoria etiquetada con _fixed_ que va poner 1 día de mora a la base de datos. Luego de esto se utiliza esta variable transitoria para calcular la mora con la fecha de consulta del usuario

In [44]:
data["DIAS_MORA"] = (data["FECHA_PAGO"] - data["FECHA_COBRO"])/np.timedelta64(1, "D")

# Definicion de columna auxiliar
data["fixem"] = 0
# Rellenar valores perdidos con un dia de mora
data.at[data["DIAS_MORA"].isna(), "fixem"] = 1
# Calculo de la mora desde la fecha consultada 
data.at[data["fixem"]==1, "DIAS_MORA"] = (data[data["fixem"]==1]["FECHA_CONSULTADA"] - data[data["fixem"]==1]["FECHA_COBRO"])/np.timedelta64(1, "D")
# Descarte de la columna provisional
data = data.drop(columns="fixem")

Cuarto campo agregado __GESTION_COBRO__. Se identifican los usuarios que tengan mas de 30 días de mora para la identificación en la gestión de cobro

In [46]:
data["GESTION_COBRO"] = (data["DIAS_MORA"] > 30).astype(int)

Quinto campo agregado __SALDADA__. Se tiene para identificar quien esta con todas sus obligaciones saldadas

In [50]:
data["SALDADA"] = (~data["FECHA_PAGO"].isna()).astype("int")

Se construye entonces la base de datos con las columnas que se van a utilizar

In [55]:
keys = ["NUMERO_CONTRATO", "TIPO_IDENTIFICACION", "IDENTIFICACION", "SEXO", "INGRESO",
        "REGION", "REPUTACION_CLIENTE", "GARANTIA_COLATERAL_CLIENTE", "NIVEL_RIESGO_CLIENTE", "CAPACIDAD_CLIENTE",
        "CIUDAD", "PROVINCIA_ESTADO_DEPARTAMENTO", "PLAZO_PACTADO", "CANTIDAD_CUOTAS_PAGADAS", "CANTIDAD_CUOTAS_PENDIENTES",
        "PORCENTAJE_INTERES_CORRIENTE_EA", "PORCENTAJE_INTERES_MORA_EA", "SALDO_CAPITAL_CONTRATO", "VALOR_INICIAL", "CANTIDAD_FACTURAS",
        "ID_CONTRATO", "ID_CLIENTE", "DIAS_MORA", "GESTION_COBRO", "DIAS_CONOCIMIENTO_FACTURA",
        "SALDADA"
       ]
data = data[keys]
data

,NUMERO_CONTRATO,TIPO_IDENTIFICACION,IDENTIFICACION,SEXO,INGRESO,REGION,REPUTACION_CLIENTE,GARANTIA_COLATERAL_CLIENTE,NIVEL_RIESGO_CLIENTE,CAPACIDAD_CLIENTE,...,PORCENTAJE_INTERES_MORA_EA,SALDO_CAPITAL_CONTRATO,VALOR_INICIAL,CANTIDAD_FACTURAS,ID_CONTRATO,ID_CLIENTE,DIAS_MORA,GESTION_COBRO,DIAS_CONOCIMIENTO_FACTURA,SALDADA
0,758589809482,5,30567722,F,2454284.0,Centro Oriente,4.0,4.0,4.0,1.0,...,18.95,6487016.0,17793000.0,62,8689.0,3929.0,-12.0,0,8.0,1
1,830910315567,2,11642038,M,5492814.0,Centro Oriente,5.0,2.0,5.0,3.0,...,22.20,14700300.0,15474000.0,4,11372.0,5614.0,-11.0,0,9.0,1
2,915320322651,3,72850794,M,5946697.0,Eje Cafetero - Antioquia,3.0,3.0,1.0,5.0,...,18.95,34162009.0,50344000.0,28,7070.0,6182.0,-14.0,0,9.0,1
3,754486263805,5,71620041,F,5489184.0,Eje Cafetero - Antioquia,5.0,1.0,5.0,5.0,...,18.95,58750314.0,62667000.0,7,10940.0,2145.0,-6.0,0,9.0,1
4,482290887380,1,46788645,F,5005455.0,Eje Cafetero - Antioquia,5.0,5.0,5.0,5.0,...,20.99,5231686.0,6439000.0,10,12811.0,3294.0,-11.0,0,9.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182774,396932713330,3,28314231,M,1806612.0,Centro Oriente,4.0,3.0,5.0,4.0,...,17.45,32809004.0,37496000.0,13,6678.0,10605.0,-13.0,0,9.0,1
182775,442296544038,4,78176770,M,609600.0,Pacífico,4.0,1.0,2.0,3.0,...,18.95,38539020.0,39359000.0,3,5952.0,8766.0,-15.0,0,9.0,1
182776,700030420895,1,68350115,M,2177628.0,Llano,5.0,1.0,2.0,1.0,...,20.99,38301835.0,40726000.0,6,12480.0,9006.0,-8.0,0,9.0,1
182777,919730907694,2,7059347,F,1539082.0,Pacífico,4.0,3.0,1.0,5.0,...,18.95,54101780.0,55847000.0,4,8604.0,797.0,-3.0,0,9.0,1


Para saber si un usuario está en mora se crea a partir de la variable __GESTION_COBRO__ una variable binaria que clasifica al usuario en dos grupos, aquel que esté en mora(1) y aquel que no(0). 

Se cambia el nombre de la variable respuesta

In [61]:
data.rename(columns={"GESTION_COBRO": "RESPONSE"})

,NUMERO_CONTRATO,TIPO_IDENTIFICACION,IDENTIFICACION,SEXO,INGRESO,REGION,REPUTACION_CLIENTE,GARANTIA_COLATERAL_CLIENTE,NIVEL_RIESGO_CLIENTE,CAPACIDAD_CLIENTE,...,PORCENTAJE_INTERES_MORA_EA,SALDO_CAPITAL_CONTRATO,VALOR_INICIAL,CANTIDAD_FACTURAS,ID_CONTRATO,ID_CLIENTE,DIAS_MORA,RESPONSE,DIAS_CONOCIMIENTO_FACTURA,SALDADA
0,758589809482,5,30567722,F,2454284.0,Centro Oriente,4.0,4.0,4.0,1.0,...,18.95,6487016.0,17793000.0,62,8689.0,3929.0,-12.0,0,8.0,1
1,830910315567,2,11642038,M,5492814.0,Centro Oriente,5.0,2.0,5.0,3.0,...,22.20,14700300.0,15474000.0,4,11372.0,5614.0,-11.0,0,9.0,1
2,915320322651,3,72850794,M,5946697.0,Eje Cafetero - Antioquia,3.0,3.0,1.0,5.0,...,18.95,34162009.0,50344000.0,28,7070.0,6182.0,-14.0,0,9.0,1
3,754486263805,5,71620041,F,5489184.0,Eje Cafetero - Antioquia,5.0,1.0,5.0,5.0,...,18.95,58750314.0,62667000.0,7,10940.0,2145.0,-6.0,0,9.0,1
4,482290887380,1,46788645,F,5005455.0,Eje Cafetero - Antioquia,5.0,5.0,5.0,5.0,...,20.99,5231686.0,6439000.0,10,12811.0,3294.0,-11.0,0,9.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182774,396932713330,3,28314231,M,1806612.0,Centro Oriente,4.0,3.0,5.0,4.0,...,17.45,32809004.0,37496000.0,13,6678.0,10605.0,-13.0,0,9.0,1
182775,442296544038,4,78176770,M,609600.0,Pacífico,4.0,1.0,2.0,3.0,...,18.95,38539020.0,39359000.0,3,5952.0,8766.0,-15.0,0,9.0,1
182776,700030420895,1,68350115,M,2177628.0,Llano,5.0,1.0,2.0,1.0,...,20.99,38301835.0,40726000.0,6,12480.0,9006.0,-8.0,0,9.0,1
182777,919730907694,2,7059347,F,1539082.0,Pacífico,4.0,3.0,1.0,5.0,...,18.95,54101780.0,55847000.0,4,8604.0,797.0,-3.0,0,9.0,1


Se evidencia como están distribuidos los morosos